**Import required libraries and scripts**

In [ ]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.ranking_functions import *
from scripts.performance_calculation import *
import numpy as np
import os

**Define initial variables**
- software : path to the software directory cloned from the git repository
- protein_file : path to the file containing the prepared protein/receptor (.pdb format)
- ref_file : path to the file containing the reference ligand used for binding pocket definition (.mol2 format)
- docking_library : path to the file containing the compounds to be used for docking (.sdf format)
- id_column : name of the column containing a unique identifier for each compound in the docking_library file (string)
- n_poses : number of to generate for each compound
- exhaustiveness : exhaustiveness of the docking algorithm (GNINA and SMINA only)

The software will then create a temporary directory to store the output of the various functions.

In [ ]:
software = '/home/tony/CADD22/software'
protein_file = '/home/tony/CADD22/wocondock_performance_testing/receptor_protoss_prepared.pdb'
ref_file = '/home/tony/CADD22/wocondock_performance_testing/crystal_ligand_protoss.sdf'
docking_library = '/home/tony/CADD22/wocondock_performance_testing/merged_actives_decoys_test.sdf'
id_column = 'Molecule Name'
n_poses = 10
exhaustiveness = 8

#Initialise variables and create a temporary folder
w_dir = os.path.dirname(protein_file)
print(w_dir)
create_temp_folder(w_dir+'/temp')

**Library preparation**

This function will first standardize the library using the ChemBL structure pipeline. This will remove salts and make the library consistent.
Then a single protonation state is predicted using pka_solver. This step may take a while depending on the number of compounds.
Finally, one conformer is generated per molecule using GypsumDL.
The final_library is then written to a file in the main directory (final_library.sdf)

In [ ]:
cleaned_df = prepare_library_GypsumDL(docking_library, id_column, software)

**Docking**

This function will dock all compounds in the receptor, using the reference ligand as a way to define the binding site. The docking results are written to the temporary folder. 

In [ ]:
all_poses = docking(protein_file, ref_file, software, exhaustiveness, n_poses)

In [ ]:
all_poses = fetch_poses(protein_file, n_poses)

**Clustering**

We will first load all the poses generated from the docking run. The cluster() function performs the calculation of the clustering metrics (for now simpleRMSD and electroshape similarity), then performs the clustering using the k-medoids clustering algorithm with the number of clusters optimised using silhouette score. Finally, all cluster centers are collected and written to a file in the temporary directory (/temp/clustering/) (one file per clustering metric).

In [ ]:
cluster(['RMSD'], w_dir, protein_file, all_poses)


In [ ]:
cluster(['espsim'], w_dir, protein_file, all_poses)
cluster(['spyRMSD'], w_dir, protein_file, all_poses)
cluster(['USRCAT'], w_dir, protein_file, all_poses)


**Rescoring**

The file containing all the cluster centers is then rescored using all scoring functions available (GNINA, Vina, AutoDock4, PLP, CHEMPLP, RF-Score-VS). The rescored output is return as a dataframe.

In [ ]:
RMSD_rescored = rescore_all(w_dir, protein_file, ref_file, software, w_dir+'/temp/clustering/RMSD_kS_full.sdf')


In [ ]:
display(RMSD_rescored)
show_correlation(RMSD_rescored)

In [8]:
espsim_rescored = rescore_all(w_dir, protein_file, ref_file, software, w_dir+'/temp/clustering/espsim_d_kS_full.sdf')
spyRMSD_rescored = rescore_all(w_dir, protein_file, ref_file, software, w_dir+'/temp/clustering/spyRMSD_kS_full.sdf')
URSCAT_rescored = rescore_all(w_dir, protein_file, ref_file, software, w_dir+'/temp/clustering/usr_kS_full.sdf')

Rescoring complete!
Rescoring with GNINA
Recommend running with single model (--cnn crossdock_default2018)
or without cnn scoring (--cnn_scoring=none).              _             
             (_)            
   __ _ _ __  _ _ __   __ _ 
  / _` | '_ \| | '_ \ / _` |
 | (_| | | | | | | | | (_| |
  \__, |_| |_|_|_| |_|\__,_|
   __/ |                    
  |___/                     

gnina v1.0 HEAD:6381355   Built Mar  6 2021.
gnina is based on smina and AutoDock Vina.
Please cite appropriately.

Commandline: ./gnina -r /home/tony/CADD22/wocondock_performance_testing/receptor_protoss_prepared.pdb -l /home/tony/CADD22/wocondock_performance_testing/temp/clustering/usr_kS_full.sdf --autobox_ligand /home/tony/CADD22/wocondock_performance_testing/crystal_ligand_protoss.sdf -o /home/tony/CADD22/wocondock_performance_testing/temp/rescoring_usr_kS_full/gnina_rescoring//rescored_crossdock_default2018.sdf --cnn crossdock_default2018 --log /home/tony/CADD22/wocondock_performance_testing/temp/rescor

21 molecules converted


rescoring progress: 1 of 1
current ligand: /home/tony/CADD22/wocondock_performance_testing/temp/rescoring_usr_kS_full/plp_rescoring//ligands.mol2 (entry 1)
LIGAND DOFs: 14
PROTEIN DOFs: 0
Simplex dimension: 14
Simplex dimension: 14
optimization finished after 0.00s
best score: -71.88


current ligand: /home/tony/CADD22/wocondock_performance_testing/temp/rescoring_usr_kS_full/plp_rescoring//ligands.mol2 (entry 2)
LIGAND DOFs: 14
PROTEIN DOFs: 0
Simplex dimension: 14
Simplex dimension: 14
optimization finished after 0.00s
best score: -52.96


current ligand: /home/tony/CADD22/wocondock_performance_testing/temp/rescoring_usr_kS_full/plp_rescoring//ligands.mol2 (entry 3)
LIGAND DOFs: 14
PROTEIN DOFs: 0
Simplex dimension: 14
Simplex dimension: 14
optimization finished after 0.00s
best score: -61.88


current ligand: /home/tony/CADD22/wocondock_performance_testing/temp/rescoring_usr_kS_full/plp_rescoring//ligands.mol2 (entry 4)
LIGAND DOFs: 14
PROTEIN DOFs: 0
Simplex dimension: 14
Simplex di






















21 molecules converted


PLANTS info: CHO hydrogen found 239 HD2 HIS246
PLANTS info: CHO hydrogen found 240 HE1 HIS246
PLANTS info: CHO hydrogen found 1028 HE1 HIS295
PLANTS info: CHO hydrogen found 2088 HE1 HIS361
PLANTS info: CHO hydrogen found 2300 HE1 HIS375
PLANTS info: CHO hydrogen found 2622 HE1 HIS396
PLANTS info: CHO hydrogen found 4117 HD2 HIS490
PLANTS info: CHO hydrogen found 4118 HE1 HIS490
rescoring progress: 1 of 1
current ligand: /home/tony/CADD22/wocondock_performance_testing/temp/rescoring_usr_kS_full/chemplp_rescoring//ligands.mol2 (entry 1)
LIGAND DOFs: 14
PROTEIN DOFs: 10
Simplex dimension: 24
Simplex dimension: 24
optimization finished after 0.00s
best score: -66.97


current ligand: /home/tony/CADD22/wocondock_performance_testing/temp/rescoring_usr_kS_full/chemplp_rescoring//ligands.mol2 (entry 2)
LIGAND DOFs: 14
PROTEIN DOFs: 10
Simplex dimension: 24
Simplex dimension: 24
optimization finished after 0.00s
best score: -45.17


current ligand: /home/tony/CADD22/wocondock_performance_testi

optimization finished after 0.00s
best score: -95.57


current ligand: /home/tony/CADD22/wocondock_performance_testing/temp/rescoring_usr_kS_full/chemplp_rescoring//ligands.mol2 (entry 21)
LIGAND DOFs: 13
PROTEIN DOFs: 10
Simplex dimension: 23
Simplex dimension: 23
optimization finished after 0.00s
best score: -72.86


total rescoring time: 1.44s
Ligands skipped due to errors: 0
Rescoring with NNScore


/home/tony/.conda/envs/wocondock/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator VarianceThreshold from version 0.24.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/tony/.conda/envs/wocondock/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/tony/.conda/envs/wocondock/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MLPRegressor from version 0.24.2 when using version 1.1.3. This might lea

Rescoring complete!


**Final ranking methods**

This code calculates the final ranking of compounds using various methods.
*Method 1* : Calculates ECR value for each cluster center, then outputs the top ranked center.
*Method 2* : Calculates ECR value for each cluster center, then outputs the average ECR value for each compound.
*Method 3* : Calculates the average rank of each compound, then ouputs the corresponding ECR value for each compound.
*Method 6* : Calculates Z-score for each cluster center, then ouputs the top ranked center.
*Method 7* : Calculates Z-score for each cluster center, then ouputs the average Z-score for each compound.

All methods are then combined into a single dataframe for comparison purposes.

In [9]:
apply_ranking_methods(w_dir)

Temporary folder was created


**Calculate enrichement factors for all methods**

In [ ]:
calculate_EFs(w_dir, docking_library)